In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from datetime import date, timedelta
import matplotlib.pyplot as plt


from Fase3_Aux_Func import (agrupamento_periodico,
                            ajuste_datas_historica,
                            conversao_array,
                            plot_serie,
                            plot_decomposicao,
                            suavizacao_simples,
                            suavizacao_exponencial,
                            holt_winters,
                            arma,
                            arima_sazonal,
                            analise_residuos,
                            check_stationarity,
                            grid_search_best_model,
                            load_models,
                            criar_forecast)

In [2]:
global local_origem 
global data_corte_treinamento

local_origem = "C:/Lab Transp/Dados Processados"
data_corte_treinamento = '2024-06-30'

### 1. Carregamento dos Dados e Arquivos dos Modelos

In [3]:
'''
LEMBRA DE MUDAR O NOME DO ARQUIVO PARA df_final_tratado DEPOIS DE AUTALIZAR O BD_BILHETAGEM E RODAR O DF TRATADO
'''
df_bilhetagem = pd.read_csv(local_origem + "/df_final_agrupado_ano_mes.csv", sep=",", decimal="," )
df_bilhetagem['Volume_Passageiros_BU_VT_DIN'] = df_bilhetagem['Volume_Passageiros_BU_VT_DIN'].astype(int)
df_bilhetagem.head()

,Ano,Mes,Zona,Sub_Prefeitura,Volume_Passageiros_BU_VT_DIN
0,2014,1,CENTRO,SE,19599200
1,2014,1,LESTE,ARICANDUVA-FORMOSA-CARRAO,1234500
2,2014,1,LESTE,ERMELINO MATARAZZO,37373
3,2014,1,LESTE,GUAIANASES,2652418
4,2014,1,LESTE,ITAIM PAULISTA,623072


In [4]:
'''
Irá realizar os ajustes de datas necessários, periorizando por dia, além de quebrar em 2 arquivos
'''
df_subpref, df_zonas = agrupamento_periodico(df_bilhetagem)

In [5]:
# Dicionário com os caminhos dos arquivos pickle dos modelos
model_file_subpref = local_origem + '/modelos/best_models_subpref_{0}.pkl'
model_file_zona =  local_origem + '/modelos/best_models_zona_{0}.pkl'

In [6]:
model_file_subpref

'/home/cdsw/modelos/best_models_subpref_{0}.pkl'

### 2. Criação dos Forecasts

In [11]:
# Loop para fazer o forecast e salvar os resultados para cada subprefeitura e zona
for sub_prefeitura in df_subpref['Sub_Prefeitura'].unique():
    output_file = f'{local_origem}/Arquivo Final/forecast_subpref_{sub_prefeitura}.csv'
    criar_forecast(df_subpref, 'Sub_Prefeitura', sub_prefeitura, model_file_subpref, output_file, 12)

print("Forecasts subprefeituras realizados e salvos com sucesso.")

Processando Sub_Prefeitura = ARICANDUVA-FORMOSA-CARRAO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_subpref_ARICANDUVA-FORMOSA-CARRAO.csv
Processando Sub_Prefeitura = BUTANTA
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_subpref_BUTANTA.csv
Processando Sub_Prefeitura = CAMPO LIMPO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_subpref_CAMPO LIMPO.csv
Processando Sub_Prefeitura = CAPELA DO SOCORRO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais

In [13]:
for zona in df_zonas['Zona'].unique():
    output_file = f'{local_origem}/Arquivo Final/forecast_zona_{zona}.csv'
    criar_forecast(df_zonas, 'Zona', zona, model_file_zona, output_file, 12)
    
print("Forecasts subprefeituras realizados e salvos com sucesso.")

Processando Zona = CENTRO
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_zona_CENTRO.csv
Processando Zona = LESTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_zona_LESTE.csv
Processando Zona = NORTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arquivo Final/forecast_zona_NORTE.csv
Processando Zona = OESTE
O intevalo corresponde a 133  datas
Séries geradas para o modelo de séries temporais
Gerando decomposição da série
Gerando função de autocorrelação
Gerando forecasts
Forecast salvo em /home/cdsw/Arqu

### 3. Consolidação dos Arquivos

In [41]:
excel_writer = pd.ExcelWriter(f'{local_origem}/Arquivo Final/Forecast_Linhas.xlsx', 
                              engine='openpyxl')

# Itere sobre todos os arquivos CSV na pasta
for arquivo_forecast in os.listdir(f'{local_origem}/Arquivo Final'):
    if arquivo_forecast.endswith('.csv'):
        
        df = pd.read_csv(os.path.join(f'{local_origem}/Arquivo Final', arquivo_forecast),       
                           sep=";",
                           decimal=",",
                           encoding='latin-1')
        
                        
        sheet_name = re.search(r'^[^_]*_[^_]*_([^.]*)', 
                               'forecast_zona_SUL.csv').group(1)
        
        df.to_excel(excel_writer, 
                    sheet_name=sheet_name,
                    index=False)

excel_writer._save()

print("Todos os arquivos CSV foram consolidados em 'consolidated_file.xlsx'.")

Todos os arquivos CSV foram consolidados em 'consolidated_file.xlsx'.
